In [1]:
1+1

2

In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, ToolMessage
from typing import TypedDict, Annotated, Sequence
import operator
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain.agents import (
    create_react_agent as create_react_agent_runnable,
)  # 변경: 함수 이름 충돌 방지
from langchain import hub
from langchain_core.runnables import RunnableLambda


# --- 환경 변수 설정 ---
load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")


# --- 기존 도구 함수들 (변경 없음) ---
def add_two(x: str) -> str:
    """숫자 입력 시 2를 더합니다."""
    try:
        return str(float(x) + 2)
    except Exception:
        raise ValueError("숫자만 입력해 주세요.")


def population_lookup(city: str) -> str:
    """도시 이름 입력 시 인구 정보를 반환합니다."""
    data = {
        "서울": "9,733,509명 (2025년 기준)",
        "부산": "3,400,000명 (2025년 기준)",
    }
    result = data.get(city.strip().lower())
    if result:
        return result
    else:
        # 이 ValueError가 fallback을 트리거하는 핵심입니다.
        raise ValueError(f"{city}의 인구 정보를 찾을 수 없습니다.")


# --- 도구 및 LLM 설정 (변경 없음) ---
tools = [
    Tool(name="AddTwo", func=add_two, description="숫자 입력 시 2를 더합니다."),
    Tool(
        name="PopulationLookup",
        func=population_lookup,
        description="도시 이름 입력 시 인구 정보를 반환합니다. 도시 이름은 '서울', '부산'만 가능합니다.",
    ),
]

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPEN_API_KEY)


# --- LangGraph 상태 정의 (변경 없음) ---
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


# --- ReAct 프롬프트 가져오기 ---
# LangGraph의 create_react_agent가 내부적으로 사용하는 것과 동일한 프롬프트를 사용합니다.
prompt = hub.pull("hwchase17/react")


# --- 노드 정의: Agent와 Action 분리 ---

# 1. Agent Node: LLM을 사용하여 다음 단계를 결정 (추론)
# 이 노드는 도구를 직접 실행하지 않고, 어떤 도구를 사용할지에 대한 결정만 내립니다.
agent_runnable = create_react_agent_runnable(llm, tools, prompt)


def run_agent(state: AgentState):
    """Agent Runnable을 실행하여 추론하고, 그 결과를 state에 추가합니다."""
    agent_output = agent_runnable.invoke(state)
    return {"messages": [agent_output]}


# 2. Action Node: Agent가 결정한 도구를 실행
# ToolNode는 도구 실행 시 발생하는 오류를 잡지 않고 상위로 전달합니다.
# 이로 인해 ValueError가 발생하면 그래프 실행이 중단되고 fallback이 트리거됩니다.
action_node = ToolNode(tools)


# --- Fallback 함수 정의 (변경 없음) ---
def create_llm_fallback():
    def llm_fallback(state: AgentState) -> dict:
        user_input = state["messages"][0].content
        prompt = f"다음 질문에 답해주세요: {user_input}"
        response = llm.invoke(prompt)
        fallback_message = f"🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:\n{response.content}"
        return {"messages": [AIMessage(content=fallback_message)]}

    return llm_fallback


# --- 라우팅 함수 정의 ---
def should_continue(state: AgentState):
    """Agent의 마지막 메시지를 보고 다음 경로를 결정합니다."""
    last_message = state["messages"][-1]
    # AIMessage에 tool_calls가 있으면 'action' 노드로 이동
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "action"
    # tool_calls가 없으면(최종 답변이면) 그래프 종료
    else:
        return END


# --- Graph 생성 및 컴파일 (수정된 로직) ---
graph = StateGraph(AgentState)

# 추론 노드와 실행 노드를 각각 추가
graph.add_node("agent", run_agent)
graph.add_node("action", action_node)

# 시작점을 'agent' 노드로 설정
graph.set_entry_point("agent")

# 조건부 엣지: 'agent' 노드 실행 후 'should_continue' 함수 결과에 따라 분기
graph.add_conditional_edges(
    "agent",
    should_continue,
    {
        "action": "action",  # 'action'을 반환하면 'action' 노드로 이동
        END: END,  # END를 반환하면 종료
    },
)

# 'action' 노드 실행 후에는 항상 다시 'agent' 노드로 돌아가서 결과를 보고 다음 행동 결정
graph.add_edge("action", "agent")

# Fallback 함수 생성
llm_fallback_runnable = RunnableLambda(create_llm_fallback())

# 그래프 컴파일 및 fallback 연결
smart_agent = graph.compile().with_fallbacks(fallbacks=[llm_fallback_runnable])


# --- 테스트 (변경 없음) ---
if __name__ == "__main__":
    print("=== 도구에서 찾을 수 있는 정보 ===")
    inputs1 = {"messages": [HumanMessage(content="서울의 인구는?")]}
    result1 = smart_agent.invoke(inputs1)
    print(result1["messages"][-1].content)

    print("\n" + "=" * 40 + "\n")

    print("=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===")
    inputs2 = {"messages": [HumanMessage(content="대구의 인구는?")]}
    result2 = smart_agent.invoke(inputs2)
    print(result2["messages"][-1].content)

=== 도구에서 찾을 수 있는 정보 ===
🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:
2023년 기준으로 서울의 인구는 약 9백만 명 정도입니다. 하지만 인구는 지속적으로 변동할 수 있으므로, 최신 통계 자료를 확인하는 것이 좋습니다. 서울특별시청이나 통계청의 공식 웹사이트에서 최신 정보를 확인할 수 있습니다.


=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===
🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:
대구의 인구는 2023년 기준으로 약 240만 명 정도입니다. 하지만 인구는 시간이 지남에 따라 변동할 수 있으므로, 최신 정보를 확인하는 것이 좋습니다. 대구의 인구에 대한 정확한 수치는 대구광역시청이나 통계청의 공식 자료를 참고하시기 바랍니다.
